In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/train_data_task1.csv
/kaggle/input/dev_data_task3.csv
/kaggle/input/train_data_task2.csv
/kaggle/input/dev_data_task1.csv
/kaggle/input/train_data_task3.csv
/kaggle/input/EXIT2024_tweet_test.csv
/kaggle/input/dev_data_task2.csv


In [41]:
!rm -rf /kaggle/working/output/best-model/*

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [2]:
!wandb offline
!wandb disabled

W&B offline. Running your script from this directory will only write metadata locally. Use wandb disabled to completely turn off W&B.
W&B disabled.


In [3]:
!pip install datasets
!pip install transformers

In [4]:
from  IPython. display import clear_output
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
import pandas as pd
import numpy as np
import random
import  matplotlib. pyplot  as  plt
from tqdm import tqdm

import torch
import torch.optim as optim
import torch. nn.functional as F
import re
import warnings
warnings.filterwarnings('ignore')

2024-05-03 08:32:13.861903: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-03 08:32:13.862037: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-03 08:32:14.118635: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [5]:
train_df = pd.read_csv('/kaggle/input/train_data_task1.csv')
test_df = pd.read_csv('/kaggle/input/EXIT2024_tweet_test.csv')
dev_df = pd.read_csv('/kaggle/input/dev_data_task1.csv')

In [7]:
test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2076 entries, 0 to 2075
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      2076 non-null   int64 
 1   lang    2076 non-null   object
 2   tweet   2076 non-null   object
 3   split   2076 non-null   object
dtypes: int64(1), object(3)
memory usage: 65.0+ KB


In [9]:
test_df.head()

,id,lang,tweet,split
0,500001,es,@Eurogamer_es Todo gamergate desde el desarrol...,TEST_ES
1,500002,es,"@ArCaNgEl__23 @Benzenazi Hombre, no es compara...",TEST_ES
2,500003,es,yo buscando las empresas metidas en el gamerga...,TEST_ES
3,500004,es,"@jordirico Primero fue internet, luego el game...",TEST_ES
4,500005,es,@AlonsoQuijano12 Yo estuve metido en el gamerg...,TEST_ES


In [6]:
def simple_preprocess(text):
    """
    pass the tweet data as a series. do not use apply function
    only preprocesses for replacing @USER and URLS
    """
    URL_RE = re.compile(r"https?:\/\/[\w\.\/\?\=\d&#%_:/-]+")
    HANDLE_RE = re.compile(r"@\w+")
    tweets = []
    for t in text:
        t = HANDLE_RE.sub("@user", t)
        t = URL_RE.sub("http", t)
        tweets.append(t)
    return tweets

In [7]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
import os.path
from os import path
from transformers import XLMRobertaForSequenceClassification, XLMRobertaTokenizer, TrainingArguments, Trainer
from datasets import Dataset
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score, f1_score
from sklearn.preprocessing import LabelEncoder

# Hard

In [8]:
#instantiate label encoders
task1_encoder = LabelEncoder()

def task1_hard_encode(df):
    task1_encoder.fit(all_task1_hard_labels)
    df['hard_label'] = task1_encoder.transform(df['hard_label'])
    return df

def task1_hard_decode(df):
    df["hard_label"] = task1_encoder.inverse_transform(df["hard_label"])
    return df

In [9]:
og_train1 = train_df.copy()
og_dev1 = dev_df.copy()
og_test1 = test_df.copy()

all_task1_hard_labels = pd.concat([og_train1["hard_label"], og_dev1["hard_label"]])
train1_df = task1_hard_encode(og_train1)

# print(train1_df.columns)
train1_df = train1_df[["tweet","hard_label"]].dropna()
train1_df = train1_df[train1_df['hard_label'] != 2]
train1_df["tweet"] = simple_preprocess(train1_df["tweet"])

dev1_df = task1_hard_encode(og_dev1)
dev1_df = dev1_df[["tweet","hard_label"]].dropna()
dev1_df = dev1_df[dev1_df['hard_label'] != 2]
dev1_df["tweet"] = simple_preprocess(dev1_df["tweet"])

test1_df = og_test1
test1_df = test1_df[["tweet"]]
# test1_df = test1_df[test1_df['hard_label'] != 2]
test1_df["tweet"] = simple_preprocess(test1_df["tweet"])

print("train1",train1_df.shape)
print(train1_df.head)
print("test1",test1_df.shape)
print(test1_df.head)

train1 (6064, 2)
<bound method NDFrame.head of                                                   tweet  hard_label
0     @user Ignora al otro, es un capullo.El problem...           1
1     @user Si comicsgate se parece en algo a gamerg...           0
2     @user Lee sobre Gamergate, y como eso ha cambi...           0
4     @user @user @user Entonces como así es el merc...           1
5     @user Aaah sí. Andrew Dobson. El que se dedicó...           0
...                                                 ...         ...
6915  idk why y’all bitches think having half your a...           1
6916  This has been a part of an experiment with @us...           1
6917  "Take me already" "Not yet. You gotta be ready...           1
6918       @user why do you look like a whore? /lh http           1
6919  ik when mandy says “you look like a whore” i l...           1

[6064 rows x 2 columns]>
test1 (2076, 1)
<bound method NDFrame.head of                                                   tweet
0     @us

In [10]:
from sklearn.model_selection import train_test_split

# Combine train1df and test1df into a single dataframe
combined_df = pd.concat([train1_df, dev1_df], ignore_index=True)

# Shuffle the combined dataframe
combined_df_shuffled = combined_df.sample(frac=1, random_state=42)

# Split the shuffled dataframe into train, validation, and test dataframes with an 80-10-10 split
train_df, val_df = train_test_split(combined_df_shuffled, test_size=0.15, random_state=42)
test_df = test1_df

# Reset the indices of the dataframes
train_df = train_df.reset_index(drop=True)
val_df = val_df.reset_index(drop=True)
test_df = test_df.reset_index(drop=True)

# with XLMB

In [11]:
device  =  torch. device('cuda:0'  if  torch. cuda. is_available() else  'cpu')
print(f"computation will run on {device} now")

computation will run on cuda:0 now


In [22]:
tokenizer = AutoTokenizer.from_pretrained('FacebookAI/xlm-roberta-base')
model = AutoModelForSequenceClassification.from_pretrained("FacebookAI/xlm-roberta-base", num_labels=2,ignore_mismatched_sizes=True)

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/615 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.10M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.12G [00:00<?, ?B/s]

Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at FacebookAI/xlm-roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [14]:
from datasets import load_dataset, DatasetDict


# Training arguments for GPT large

https://datascience.stackexchange.com/questions/64583/what-are-the-good-parameter-ranges-for-bert-hyperparameters-while-finetuning-it

In [24]:
# Set parameters
MAX_LENGTH = 512

# Define the training arguments
training_args = TrainingArguments(
    output_dir="./output/best-model",
    report_to=None,
    num_train_epochs=5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    learning_rate=2e-5,
    weight_decay=0.01,
    evaluation_strategy="steps",
    save_total_limit = 1,
    logging_strategy="epoch",
    load_best_model_at_end=True,
)

In [16]:
def convert_to_dataset(df):
    df = {"text": df['tweet'].tolist(), "label": df["hard_label"].tolist()}
    dataset = Dataset.from_dict(df)
    return dataset

def convert_to_dataset_test(df):
    df = {"text": df['tweet'].tolist()}
    dataset = Dataset.from_dict(df)
    return dataset

In [25]:
# Convert dataframe to dataset
train_dataset = convert_to_dataset(train_df)
val_dataset = convert_to_dataset(val_df)
test_dataset = convert_to_dataset_test(test_df)

# Create the datasets
train_encodings = tokenizer(train_dataset["text"],return_tensors='pt', padding='max_length', truncation=True, max_length=MAX_LENGTH)
train_dataset = Dataset.from_dict({"input_ids": train_encodings["input_ids"], "attention_mask": train_encodings["attention_mask"], "label": train_dataset["label"]})

val_encodings = tokenizer(val_dataset["text"], padding='max_length', truncation=True, max_length=MAX_LENGTH)
val_dataset = Dataset.from_dict({"input_ids": val_encodings["input_ids"], "attention_mask": val_encodings["attention_mask"], "label": val_dataset["label"]})


test_encodings = tokenizer(test_dataset["text"], padding='max_length', truncation=True, max_length=MAX_LENGTH)
test_dataset = Dataset.from_dict({"input_ids": test_encodings["input_ids"], "attention_mask": test_encodings["attention_mask"]})

In [18]:
def compute_f1_score(pred):
    # pred is a tuple (predictions, labels)
    predictions, labels = pred
    # Compute the F1 score
    f1 = f1_score(labels, predictions.argmax(axis=1), average='macro')
    return {"f1_score": f1}

optimizers (Tuple[torch.optim.Optimizer, torch.optim.lr_scheduler.LambdaLR], optional, defaults to (None, None)) — A tuple containing the optimizer and the scheduler to use. Will default to an instance of AdamW on your model and a scheduler given by get_linear_schedule_with_warmup() controlled by args.

# with lr=2e-5

In [26]:
# model.to(device)
# Define the trainer for each fold
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_f1_score
)

# Train the model for each fold
trainer.train()

Step,Training Loss,Validation Loss,F1 Score
500,0.519100,0.427689,0.822765
1000,0.370700,0.449140,0.836119
1500,0.200700,0.560502,0.836953


TrainOutput(global_step=1860, training_loss=0.30053384637319913, metrics={'train_runtime': 1965.0157, 'train_samples_per_second': 15.135, 'train_steps_per_second': 0.947, 'total_flos': 7824922786406400.0, 'train_loss': 0.30053384637319913, 'epoch': 5.0})

# with lr=2e-5

In [30]:
from scipy.special import softmax
from sklearn.metrics import accuracy_score, f1_score

# Get predictions on the validation set
val_predictions = trainer.predict(val_dataset)
val_pred_labels = np.argmax(val_predictions.predictions, axis=1)
val_true_labels = val_dataset["label"]

val_accuracy = accuracy_score(val_true_labels, val_pred_labels)
val_f1_score = f1_score(val_true_labels, val_pred_labels)

print("Validation Accuracy:", val_accuracy)
print("Validation F1 Score:", val_f1_score)

Validation Accuracy: 0.8228571428571428
Validation F1 Score: 0.8268156424581005


In [31]:
# Get predictions on the test set
from scipy.special import softmax
test_predictions = trainer.predict(test_dataset)
test_pred_labels = np.argmax(test_predictions.predictions, axis=1)
test_pred_probs = softmax(test_predictions.predictions).tolist()

#get probabilities to a list
test_probs_list = [] 
for logits in test_pred_probs:
    test_probs_list.append({'YES': logits[0], 'NO': logits[1]})

In [32]:
import json
def get_json_for_evaluation(df, probs_list, ids, gold):
    gold["index"] = gold["id"].astype(int)
    output_dict = {}
    decoded_labels = task1_hard_decode(df)["hard_label"].tolist()
    print("decoded labels length:",len(decoded_labels))
    for i, row in enumerate(probs_list):
        soft_label = row
        hard_label = decoded_labels[i]
        local_dict = {"hard_label":hard_label,"soft_label": soft_label}
        output_dict[int(gold["index"][i])] = local_dict
    
    filename = "./output/task1_NICA-hard-8.json"
    print("output generated as json")
    with open(filename, 'w') as file:
        json.dump(output_dict, file, indent=4)
    return output_dict

outputs_json = get_json_for_evaluation(pd.DataFrame(test_pred_labels,columns=["hard_label"]), test_probs_list, og_test1[["id"]],og_test1)

decoded labels length: 2076
output generated as json
